In [ ]:
# presteps: install packages and restart the kernel
!pip install statsmodels

In [ ]:
from datetime import datetime, date
import os 
from IPython.display import display, HTML
import pandas as pd
import numpy, scipy
import matplotlib, matplotlib.pyplot as plt
from matplotlib import rcParams
from datetime import date
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import laplace
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
import joblib
from copy import deepcopy

bucket = os.getenv("WORKSPACE_BUCKET") # my work dir
dataset = os.getenv("WORKSPACE_CDR")
genomic_location = os.getenv("CDR_STORAGE_PATH")

print(bucket)
print(dataset)
print(genomic_location)

# PLINK Quality Control

In [ ]:
# copy files from bucket to local
!gsutil -u $GOOGLE_PROJECT cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv .
!gsutil -u $GOOGLE_PROJECT cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness/relatedness_flagged_samples.tsv .
!gsutil -u ${GOOGLE_PROJECT} ls gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel
    
!mkdir -p plink2
!gsutil -u $GOOGLE_PROJECT -m cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/clinvar_v7.1/plink_bed/* plink2/

In [ ]:
%%bash
for chr in {13,14,20,21,4,8}; do 
    plink --bfile plink2/clinvar.chr${chr} \
          --maf 0.01 \
          --geno 0.05 \
          --mind 0.05 \
          --hwe 1e-6 \
          --recode AD \
          --out genotype_data_qc${chr}
done

# Load genotype data


In [ ]:
chrs = [13,14,20,21,4,8]
dfs = []
dfs_filter = []
relatedness = pd.read_csv("relatedness_flagged_samples.tsv", delimiter="\t")
print(relatedness.shape)
relatedness.columns = ['IID']
for chr_ in chrs:
    print(f"Chr Data reading {chr_}...")
    df = pd.read_csv(f'./genotype_data_qc{chr_}.raw', delimiter=' ')
    df_filtered = df[~df['IID'].isin(relatedness['IID'].values.tolist())]
    print(df.shape)
    print(df_filtered.shape)
    dfs.append(df)
    dfs_filter.append(df_filtered)

In [ ]:
# Merge processed data for all chromosomes

common_iids = set(dfs_filter[0]['IID'])
for df in dfs_filter[1:]:
    common_iids.intersection_update(df['IID'])
    
common_iids = list(common_iids)
print(len(common_iids))

filtered_dfs = [df[df['IID'].isin(common_iids)].reset_index(drop = True) for df in dfs_filter]

# snp data
snp_data = pd.concat([item.iloc[:, 6:] for item in filtered_dfs], axis = 1)
print(snp_data.shape)

# meta data
meta_data = filtered_dfs[0].loc[:, ['IID', 'SEX', 'FID']]
print(snp_data.shape, meta_data.shape)

# final data
final_data = pd.concat([meta_data, snp_data], axis = 1)
print(final_data.shape)

final_data.head()

In [ ]:
# Merge ancestry data

start = time.time()
ancestry_pred = pd.read_csv("ancestry_preds.tsv", delimiter="\t")
final_data = pd.merge(
    final_data.set_index('IID'), ancestry_pred.set_index('research_id')[['ancestry_pred']], 
    how = 'inner', left_index = True, right_index = True
)
final_data = final_data.drop(['SEX', 'FID'], axis = 1)
columns = final_data.columns[-1:].tolist() + final_data.columns[:-1].tolist()
final_data = final_data[columns]

final_data.rename(columns={'ancestry_pred': 'population'},inplace = True)
end = time.time()
print(f'{end - start}s')
final_data.to_csv('final_data_filtered.csv', index = False)
final_data.head()

# Generate Synthetic SNPs


In [ ]:
def preprocessing(X:np.array, k = 6):
    '''
    Preprocessing SNPs data using standard scaler and PCA and get Kmeans model
    
    X: (n_users, n_snps) - numpy array
    k: number of clusters
    '''
    # fill missing data with 0
    X[np.isnan(X)] = 0
    print(X.shape)

    # standard scaler
    start = time.time()
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    end = time.time()
    print(f'{end - start}s')

    # pca on whole data
    start = time.time()
    pca = PCA(n_components=d, random_state=seed)
    X_pca = pca.fit_transform(X)
    print(X_pca.shape)
    end = time.time()
    print(f'{end - start}s')
    joblib.dump(pca, 'pca_model.pkl')

    # kmeans 
    start = time.time()
    kmeans = KMeans(n_clusters=k, random_state=seed, n_init = 'auto')
    kmeans.fit(X_pca)
    end = time.time()
    print(f'{end - start}s')
    joblib.dump(kmeans, 'kmeans_model.pkl')
    
    return scaler, pca, kmeans, X_pca

def randomized_response(val, p, q):
    '''
    Add randomized response to the SNPs data
    
    val: int
    p: float
    q: float
    '''
    rand_val = np.random.uniform(0, 1)
    new_val = val
    if rand_val > p:
        if val == 0:
            new_val = 1
        elif val == 2:
            new_val = 1
        elif val == 1:
            if rand_val > p+q:
                new_val = 0
            else:
                new_val = 2
    return new_val


def add_noise_shuffle(X_sample: np.array, p, q, seed = 21):
    '''
    Add randomized response to the SNPs data by the following steps:
    1. Generate a random uniform random array
    2. Apply randomized response to the SNPs data
    3. Handle 0 and 2 with probability p to be 1
    4. Handle 1 with probability p+q to be 0 and 1 - p - q to be 2
    
    X_sample: (n_users, n_snps) - numpy array
    p: float
    q: float
    seed: int
    '''
    np.random.seed(seed)
    
    # random uniform random array
    rand_matrix = np.random.uniform(0, 1, size=X_sample.shape)
    
    # generated matrix
    X_sample_noise = X_sample.copy()
    
    # apply randomized response
    
    # handle 0 and 2
    mask_0_2 = (X_sample == 0) | (X_sample == 2)
    X_sample_noise[mask_0_2 & (rand_matrix > p)] = 1
    
    # handle 1
    mask_1 = X_sample == 1
    X_sample_noise[mask_1 & (rand_matrix > p) & (rand_matrix > p+q)] = 0
    X_sample_noise[mask_1 & (rand_matrix > p) & (rand_matrix <= p+q)] = 2
    
    return X_sample_noise

def validation(X_sample_real, X_sample_gen, scaler, pca, kmeans):
    '''
    Validation of the generated SNPs data
    1. Equality check - it should not be the same as the real data
    2. Clustering check - it should fall into the same cluster as the real data
    
    X_sample_real: (n_users, n_snps) - numpy array
    X_sample_gen: (n_users, n_snps) - numpy array
    scaler: sklearn.preprocessing.StandardScaler
    pca: sklearn.decomposition.PCA
    kmeans: sklearn.cluster.KMeans
    '''
    
    # equality check
    equal_compare = np.all(X_sample_real == X_sample_gen, axis = 1)    
    
    # clustering check
    #     X_sample_gen_pca = pca.transform(scaler.transform(X_sample_gen))
    #     X_sample_real_pca = pca.transform(scaler.transform(X_sample_real))
    X_sample_gen_pca = pca.transform(X_sample_gen)
    X_sample_real_pca = pca.transform(X_sample_real)
    cluster_compare = (kmeans.predict(X_sample_gen_pca) != kmeans.predict(X_sample_real_pca))
    
    compare = np.array([equal_compare, cluster_compare])
    low_quality_ids = np.where(compare.transpose().any(axis=1))[0]
    
    return low_quality_ids

def process_generated_data(gen_results):
    '''
    Process the generated SNPs data - merge together generated snps and sampling
    
    gen_results: dict   
    '''
    threshold = 5000

    gen_results_old = deepcopy(gen_results)
    final_gen_datas = []

    for population_name, ret in gen_results_old.items():

        gen_datas = []
        for item in ret:
            gen_datas.append(item['X_gen_final'])

        gen_data = np.concatenate(gen_datas, axis = 0)
        gen_data = pd.DataFrame(gen_data)
        print(population_name, gen_data.shape)
        # sampling
        gen_data = gen_data.sample(n = threshold, random_state = 32).reset_index(drop = True)
        gen_data['population'] = population_name
        print(gen_data.shape)

        final_gen_datas.append(gen_data)

    final_gen_df = pd.concat(final_gen_datas, axis = 0)
    
    final_gen_df['sample_id'] = np.arange(1, final_gen_df.shape[0]+1)
    columns = ['sample_id', 'population'] + final_gen_df.columns.tolist()[:-2]
    final_gen_df = final_gen_df[columns]
    print(final_gen_df.shape)
    
    return final_gen_df

## Main script for generating synthetic SNPs


In [ ]:
# parameters
d = 200
num_sample_threshold = 5000
seed = 21
population_name = 'eur'
epsilon = 60
eps = epsilon / 10.0
p = np.exp(eps)/(np.exp(eps)+2)
q = 1/(np.exp(eps)+2)
print(p, q)

# Preprocessing
seed = 21
X = final_data.iloc[:, 1:].values.copy()
k = final_data['population'].nunique()
scaler, pca, kmeans, X_pca = preprocessing(X)

# Generation - for each population, repeat generation multiple times until number of valid samples reach the sample threshold
gen_results = {}
population_group = final_data['population'].value_counts().to_dict()
for population_name, population_count in population_group.items():
    
    ##########################################################################################################
    # Generation for each population
    ##########################################################################################################

    sample_ids = final_data[final_data['population'] == population_name].reset_index(drop=True).index.tolist()
    sample_ids_copy = sample_ids.copy()
    print(population_name, population_count, len(sample_ids))
    
    total_num_samples = 0
    run_times = 5
    repetition_iter = 1
    total_rets = []
    while total_num_samples <= num_sample_threshold:

        print('='*50)
        print(f'Repetition {repetition_iter} ...')
        repetition_iter += 1
        seed = (seed + 10986)%((2^31)-1)
        sample_ids = sample_ids_copy

        ################################################################################
        # Generation for one repetition
        X_gens = []
        sample_ids_list = []
        low_quality_ids_list = []
        num_valid_samples = 0

        for iteration in range(run_times):

            print(f'Times: {iteration}')

            # generation
            X_real_sample = final_data.iloc[:, 1:].values[sample_ids, :]
            X_gen_sample = add_noise_shuffle(X_real_sample, p, q, seed = seed)

            print(X_gen_sample.shape, X_real_sample.shape)

            # validation
            low_quality_id_indices = validation(X_gen_sample, X_real_sample, scaler, pca, kmeans)
            sample_ids_array = np.array(sample_ids)
            low_quality_ids = sample_ids_array[low_quality_id_indices].tolist()
            print(len(low_quality_ids))

            X_gens.append(X_gen_sample)
            sample_ids_list.append(sample_ids)
            low_quality_ids_list.append(low_quality_ids)

            num_valid_samples += len(sample_ids) - len(low_quality_id_indices)
            
            if num_valid_samples > 5000 or len(low_quality_ids) == 0:
                break

            sample_ids = low_quality_ids # regenrate snps for low quality ids
            seed = (seed + 10986)%((2^31)-1)

        #############################################################################
        # craft generated samples
        assert len(X_gens[0]) == len(sample_ids_list[0])
        X_gen_sample_final = X_gens[0]
        for sample_ids_item, X_gen_item in zip(sample_ids_list[1:], X_gens[1:]):
            X_gen_sample_final[sample_ids_item, :] = X_gen_item

        X_gen_sample_final = np.delete(X_gen_sample_final, low_quality_ids_list[-1], axis = 0)
        sample_ids_final = list(set(sample_ids_list[0]).difference(set(low_quality_ids_list[-1])))

        total_num_samples += num_valid_samples
        print(f'Current total sample: {total_num_samples}, current round gen data shape {X_gen_sample_final.shape} ({len(sample_ids_final)})')

        total_rets.append({
            'X_gens': X_gens,
            'sample_ids_list': sample_ids_list,
            'low_quality_ids_list': low_quality_ids_list,
            'X_gen_final': X_gen_sample_final.copy()
        })

    print(total_num_samples)
    gen_results[population_name] = total_rets
    
final_gen_df = process_generated_data(gen_results)
final_gen_df.head()

In [ ]:
final_gen_df.to_csv('./allofus_snps.csv', index = False)
final_data.reset_index(drop = True).to_csv('./allofus_snps_real.csv', index = False)

# Phenotype Simulation

In [ ]:
'''
The following script need to be run in R envronment, First change the kernel to R and then run the following script
'''

if (!require("BiocManager", quietly = TRUE))
 install.packages("BiocManager")

BiocManager::install("snpStats")

install.packages('PhenotypeSimulator')
install.packages('tidyverse')
install.packages('data.table')

library(tidyverse)
library(PhenotypeSimulator)
file_path <- "./allofus_snps.csv"
snp_data <- read_csv(file_path)
genotypes = snp_data %>% select(3:ncol(.)) %>% as.matrix()

noiseBg <- noiseBgEffects(N = nrow(genotypes), P = 1)
causalSNPs <- getCausalSNPs(N=nrow(genotypes), NrCausalSNPs = round(0.1*ncol(genotypes)),genotypes=genotypes)
causalSNPs <- standardiseGenotypes(genotypes)
geneticFixed <- geneticFixedEffects(N=nrow(genotypes), X_causal=causalSNPs, P=1, id_samples = 1:nrow(genotypes))

genVar <- 0.9
noiseVar <- 1- genVar
# rescale phenotype components
genBg_ind_scaled <- rescaleVariance(geneticFixed$independent, genVar)
noiseBg_ind_scaled <- rescaleVariance(noiseBg$independent, noiseVar)

# combine components into final phenotype
Y <- scale(genBg_ind_scaled$component + noiseBg_ind_scaled$component)

# transform to binary
sigmoid <- function(x) {
  1 / (1 + exp(-x))
}

Y_sigmoid <- sigmoid(Y)
threshold <- median(Y_sigmoid)
Y_binary <- as.integer(Y_sigmoid >= threshold)

pheno_data <- snp_data %>%
    select(SampleID) %>%
    mutate(PhenotypeCondition = Y_binary)

write_csv(pheno_data, "./allofus_phenotype.csv", col_names = TRUE)

hist(Y_binary)

# Generate Watermark SNPs

In [ ]:
def generate_synthetic_snp(n, phenotype_vector, eps, z0):
    """
    Generate a synthetic SNP vector based on the provided phenotype vector.
    
    Parameters:
    - n: The number of subjects (size of the phenotype vector)
    - phenotype_vector: A numpy array of size n containing the phenotype values (0s and 1s)
    - eps: Epsilon parameter for controlling the noise
    - z0: Proportion factor for 0s and 1s in the phenotype vector
    
    Returns:
    - synthetic_snp: A synthetic SNP vector with values in {0, 1, 2}
    """
    
    # Step 1: Initialize the synthetic SNP vector
    synthetic_snp = phenotype_vector.copy()
    
    # Step 2: Find indices where phenotype is 0 (i0) and 1 (i1)
    i0 = np.where(phenotype_vector == 0)[0]
    i1 = np.where(phenotype_vector == 1)[0]
    
    # Step 3: Compute the number of 1's to be removed (or added if negative)
    nremove1 = len(i1) - int(n * z0 * (1 - eps))
    
    # Step 4: Compute the number of 0's to be removed (or added if negative)
    nremove0 = int(n * eps) - nremove1
    
    # Step 7: Randomly assign selected entries of s* to 2
    # Handle cases based on nremove1 and nremove0
    if nremove1 > 0 and nremove0 > 0:  # Case a
        # Randomly pick nremove1 indices from i1, and nremove0 from i0, set corresponding values to 2
        selected_i1 = np.random.choice(i1, size=nremove1, replace=False)
        selected_i0 = np.random.choice(i0, size=nremove0, replace=False)
        synthetic_snp[selected_i1] = 2
        synthetic_snp[selected_i0] = 2
    
    elif nremove1 <= 0 and nremove0 > 0:  # Case b
        # Randomly pick nremove0 indices from i0, and set values to 2
        selected_i0 = np.random.choice(i0, size=nremove0, replace=False)
        # Randomly pick |nremove1| indices from i1, set them to 1, then set others to 2
        selected_i1 = np.random.choice(i1, size=abs(nremove1), replace=False)
        synthetic_snp[selected_i1] = 1
        synthetic_snp[selected_i0] = 2
    
    elif nremove1 > 0 and nremove0 <= 0:  # Case c
        # Randomly pick nremove1 indices from i1, and set values to 2
        selected_i1 = np.random.choice(i1, size=nremove1, replace=False)
        # Randomly pick |nremove0| indices from i0, set them to 0, then set others to 2
        selected_i0 = np.random.choice(i0, size=abs(nremove0), replace=False)
        synthetic_snp[selected_i1] = 2
        synthetic_snp[selected_i0] = 0
    
    return synthetic_snp

def calculate_pvalues_logit(X, y):
    '''
    Calculate p-values for the logistic regression model
    
    X: (n_users, n_snps) - numpy array
    y: (n_users) - numpy array
    '''
    # Add a constant term to the features matrix
    X = sm.add_constant(X)
    
    # Fit the logistic regression model
    model = sm.Logit(y, X).fit()
    
    # Extract p-values
    return model.pvalues[1:]  # Exclude the constant term

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

gen_data = pd.read_csv("./allofus_phenotype.csv")
gen_data.head()

# Generate Watermark SNPs
phenotype = gen_data['PhenotypeCondition']
n_watermark = 20
n_samples = phenotype.shape[0]

watermark = np.zeros((n_samples, n_watermark))
for i in range(n_watermark):
    watermark[:, i] = generate_synthetic_snp(n = phenotype.shape[0], phenotype_vector=phenotype, eps = 0.3, z0 = 0.49)

# logistic regression - p-value - significant
# Calculate p-values
p_values = calculate_pvalues_logit(watermark, phenotype)

# Print raw p-values
print("Raw p-values:")
print(p_values)
print(p_values.mean(), p_values.std())

watermark_data = np.concatenate([watermark, p_values.to_numpy().reshape(1,-1)], axis = 0)
print(watermark_data.shape)
watermark_df = pd.DataFrame(watermark_data, columns = [f'Watermark_SNP_{i}' for i in range(n_watermark)])
index = phenotype_data['SampleID'].astype(str).values.tolist() + ['p-value']
watermark_df.index = index

watermark_df.to_csv('./allofus_watermarksnp.csv', index = True)

# Generate Synthetic Relatives

In [ ]:
import pandas as pd
import numpy as np
import random

def add_child(df, childID, fatherID, motherID):
    '''
    Add child SNPs data to the dataframe
    
    df: (n_users, n_snps) - pandas DataFrame
    childID: str - child ID
    fatherID: str - father ID
    motherID: str - mother ID
    '''
    fatherCol = df.loc[:, fatherID].to_numpy()
    motherCol = df.loc[:, motherID].to_numpy()
    
    # Initialize the child's SNP column with zeros
    childCol = np.zeros_like(fatherCol)
    
    # Logic for SNP inheritance based on father and mother SNP values
    for i in range(len(fatherCol)):
        p = random.uniform(0, 1)
        if fatherCol[i] == 0 and motherCol[i] == 0:
            childCol[i] = 0
        elif (fatherCol[i] == 0 and motherCol[i] == 1) or (fatherCol[i] == 1 and motherCol[i] == 0):
            childCol[i] = 0 if p > 0.5 else 1
        elif (fatherCol[i] == 0 and motherCol[i] == 2) or (fatherCol[i] == 2 and motherCol[i] == 0):
            childCol[i] = 1
        elif fatherCol[i] == 1 and motherCol[i] == 1:
            if p < 0.333:
                childCol[i] = 0
            elif p < 0.666:
                childCol[i] = 1
            else:
                childCol[i] = 2
        elif (fatherCol[i] == 1 and motherCol[i] == 2) or (fatherCol[i] == 2 and motherCol[i] == 1):
            childCol[i] = 1 if p > 0.5 else 2
        elif fatherCol[i] == 2 and motherCol[i] == 2:
            childCol[i] = 2
    
    # Add the child's SNP data to the DataFrame
    df[childID] = childCol
    return df

def create_relatives(Gdata, n = 400):
    '''
    Create synthetic relatives
    
    Gdata: (n_snps, n_users) - numpy array
    n: int - number of relatives
    '''
    rel_ids = []
    
    Gdata_n = Gdata.T.copy()
    parent_ids = np.random.choice(Gdata_n.columns, size=n, replace=False)
    relative_mapping = {}
    # Step 1: 1st generation
    F_id = parent_ids[:n//2]  # Father IDs
    M_id = parent_ids[n//2:]  # Mother IDs
    child_ids_1C = ['1C_' + str(i) for i in range(n//2)]  # Generate 100 child IDs
    for c_id, f_id, m_id in zip(child_ids_1C, F_id, M_id):
        Gdata_n = add_child(Gdata_n, c_id, f_id, m_id)
        relative_mapping[c_id] = [f_id, m_id]
        for ances_id in relative_mapping[c_id]:
            rel_ids.append([c_id, ances_id, 'first-degree'])

    # Step 2: 2nd generation
    np.random.shuffle(child_ids_1C)  # Shuffle the first-degree children
    group_1CF = child_ids_1C[:n//4]  # 50 from the first group
    group_1CM = child_ids_1C[n//4:]  # 50 from the second group
    child_ids_2C = ['2C_' + str(i) for i in range(n//4)]
    for c_id, f_id, m_id in zip(child_ids_2C, group_1CF, group_1CM):
        Gdata_n = add_child(Gdata_n, c_id, f_id, m_id)
        f_ances_ids = relative_mapping[f_id]
        m_ances_ids = relative_mapping[m_id]
        relative_mapping[c_id] = f_ances_ids + m_ances_ids
        for ansces_id in relative_mapping[c_id]:
            rel_ids.append([c_id, ansces_id, 'second-degree'])

    # Step 3: 3rd generation
    np.random.shuffle(child_ids_2C)  # Shuffle the second-degree children
    group_2CF = child_ids_2C[:n//8]
    group_2CM = child_ids_2C[n//8:]
    child_ids_3C = ['3C_' + str(i) for i in range(n//8)]
    for c_id, f_id, m_id in zip(child_ids_3C, group_2CF, group_2CM):
        Gdata_n = add_child(Gdata_n, c_id, f_id, m_id)
        f_ances_ids = relative_mapping[f_id]
        m_ances_ids = relative_mapping[m_id]
        for ansces_id in f_ances_ids + m_ances_ids:
            rel_ids.append([c_id, ansces_id, 'third-degree'])

    # Step 6: Create the relationship DataFrame and save it to a CSV file
    rel_ids_df = pd.DataFrame(rel_ids, columns=["sampleID", "ancestryID", "generation"])

    return Gdata_n.T, rel_ids_df

def calculate_kinship(genotype_i: np.ndarray, genotype_j: np.ndarray) -> float:
    """
    Calculates the KING kinship coefficient between two individuals.

    Parameters:
    genotype_i (np.ndarray): Genotype data of the first individual.
    genotype_j (np.ndarray): Genotype data of the second individual.

    Returns:
    float: The calculated KING kinship coefficient.
    """
    n11 = np.sum((genotype_i == 1) & (genotype_j == 1))
    n02 = np.sum((genotype_i == 2) & (genotype_j == 0))
    n20 = np.sum((genotype_i == 0) & (genotype_j == 2))
    n1_s = np.sum(genotype_i == 1)
    s_1 = np.sum(genotype_j == 1)

    if n1_s == 0:
        return 0
    
    phi_ij = (2 * n11 - 4 * (n02 + n20) - n1_s + s_1) / (4 * n1_s)
    
    if phi_ij > 0.5:
        return 0.5
    
    if phi_ij < 0:
        return 0
    
    return phi_ij

In [ ]:
gen_data = pd.read_csv("./allofus_snps.csv")
population_dict = gen_data['population'].value_counts().to_dict()

import warnings
warnings.filterwarnings("ignore")

# parameters
n_samples = 200

# gen synthetic relatives
gen_relatives = {}
for population_name, _ in population_dict.items():
    print(population_name)
    
    gdata = gen_data[gen_data['population'] == population_name].set_index('SampleID').iloc[:, 1:].copy()
    print(gdata.shape)
    gdata_n, rel_ids_df = create_relatives(gdata, n_samples*2)
    print(gdata_n.shape, rel_ids_df.shape)
    
    gen_relatives[population_name] = {
        'gdata': gdata_n.reset_index(),
        'rel': rel_ids_df
    }
    
# compute kinship
results = []
for name, item in gen_relatives.items():
    print("======================================================================")
    print(name)

    kinship_df = item['rel']
    genotype_df = item['gdata']

    def get_kinship(row):
        c_geno = genotype_df[genotype_df['SampleID'] == row['sampleID']].iloc[0, 2:]
        a_geno = genotype_df[genotype_df['SampleID'] == row['ancestryID']].iloc[0, 2:]

        return calculate_kinship(c_geno, a_geno)
    
    kinship_df['kinship'] = kinship_df.apply(lambda row: get_kinship(row), axis = 1)
    results.append(kinship_df)

result_df = pd.concat(results, axis = 0)
result_df.head()

In [ ]:
gen_relatives['eur']['rel']

In [ ]:
gen_relatives['eur']['gdata']

In [ ]:
total_relatives = n_samples + n_samples//2 + n_samples//4
print(total_relatives)

relative_snps_datas = [item['gdata'].iloc[-total_relatives:, :] for item in gen_relatives.values()]

relative_snp_data = pd.concat(relative_snps_datas, axis = 0)

print(relative_snp_data.shape)
relative_snp_data.to_csv('./allofus_relative_snps.csv', index = False)
relative_snp_data.head()

In [ ]:
'''
Validate Relative SNPs by Kinship
'''

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

result_df['generation'] = result_df['generation'].str.split('-', expand = True)[0]

plt.rcParams['text.usetex'] = False
plt.rcParams.update({'figure.dpi': '300'})

# Set up the plot style
plt.figure(figsize=(8, 4))
sns.set_style("white")
sns.set_context("notebook", font_scale=1.1)

# Create the histogram
sns.histplot(
    data=result_df, x='kinship', hue='generation', kde=False, multiple = 'layer',palette='tab10', stat = 'count',bins = 50,
    hue_order=['third','first', 'second']
)
# Show the plot
plt.xlim(0, 0.5)
plt.xlabel('')
plt.ylabel('')
plt.tight_layout()
plt.grid(False)
plt.show()


# Analysis and Visualization


In [ ]:
final_gen_df = pd.read_csv('./allofus_snps.csv')
print(final_gen_df['population'].value_counts(normalize = True))
final_gen_df.head()

In [ ]:
final_df = pd.read_csv('./allofus_snps_real.csv')
final_df['population'].value_counts(normalize = True)

## MAF Comparison

In [ ]:
def verfication(gen_data, verbose = 1000):
    '''
    MAF Comparison between synthetic and real data
    '''
    
    df = gen_data
    # Assuming 'df' is your DataFrame
    # Identify SNP columns (assuming they start with 'SNP')
    snp_columns = df.columns.tolist()[2:]
    print("#"*100)
    #################################################################################
    # Initialize a dictionary to store MAF values
    maf_dict = {}

    # Iterate over each SNP column
    for idx, snp in enumerate(snp_columns):
        # Get counts of each genotype (0, 1, 2)
        genotype_counts = df[snp].value_counts()

        # Extract counts, defaulting to 0 if a genotype is absent
        n0 = genotype_counts.get(0, 0)  # Homozygous major allele
        n1 = genotype_counts.get(1, 0)  # Heterozygous
        n2 = genotype_counts.get(2, 0)  # Homozygous minor allele

        # Calculate total number of alleles (2 per individual)
        total_alleles = 2 * (n0 + n1 + n2)

        # Calculate counts of each allele
        major_allele_count = (2 * n0) + n1
        minor_allele_count = (2 * n2) + n1

        # Calculate allele frequencies
        major_allele_freq = major_allele_count / total_alleles
        minor_allele_freq = minor_allele_count / total_alleles

        # MAF is the frequency of the minor allele
        maf = min(major_allele_freq, minor_allele_freq)

        # Store the MAF value
        maf_dict[snp] = maf
        
        if idx <= verbose:
            print("MAF:", maf)

    # Convert the MAF dictionary to a DataFrame for better visualization
    maf_df = pd.DataFrame.from_dict(maf_dict, orient='index', columns=['MAF'])
    print("#"*100)
    #################################################################################
    # Print Statistics
    distribution = {}
    for idx, col in enumerate(gen_data.columns[2:]):
        distribution[col] = gen_data.loc[:, col].value_counts().to_dict()
        if idx < verbose:
            print(idx, distribution[col])
    
    return maf_df, distribution

maf_df, distribution_dict = verfication(final_gen_df, verbose = 5000)
maf_df.hist(bins = 100)

In [ ]:
maf_df, distribution_dict = verfication(final_gen_df, verbose = 5000)
maf_df_real, distribution_dict_real = verfication(final_df, verbose = 100)
fig, ax = plt.subplots(1,1,figsize = (6,5))
plt.hist(maf_df.values, label = 'synthetic', bins  = 100, alpha = 0.8)
plt.hist(maf_df_real.values, label = 'original', bins = 100, alpha = 0.8)
plt.grid(True)
plt.show()

## TSNE Visualization

Conduct TSNE Visualization to verfiy the distribution of synthetic data and real data for each population


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD

def perform_tsne(X, scaler = 'standard', n_components_pca = 20, method = 'pca', 
                 perplexity = 30, learning_rate = 200, n_iter = 1000, seed = 21):
    
    #X = X.astype(int)
    
    if scaler == 'one-hot':
        X = X.astype(int)
        oh = OneHotEncoder(sparse_output = False)
        X_scaled = oh.fit_transform(X)
        #X_oh = sparse.csr_matrix(X_oh)
    else:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

    # Dimensionality reduction using PCA or TruncatedSVD
    start = time.time()
    if method == 'pca':
        reducer = PCA(n_components=n_components_pca, random_state=seed)
    elif method == 'svd':
        reducer = TruncatedSVD(n_components=n_components_pca, random_state=seed)
    else:
        raise ValueError("Method must be 'pca' or 'svd'")

    X_reduced = reducer.fit_transform(X_scaled)

    # Apply t-SNE
    tsne = TSNE(n_components=2,random_state=seed, n_jobs=-1, learning_rate=learning_rate, perplexity=perplexity, n_iter=n_iter,
               verbose = 0)
    X_tsne = tsne.fit_transform(X_reduced)
    return X_tsne

In [ ]:
# tnse visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import time
import gower

# parameters
gen_data = final_gen_df
d = 100
sample_ratio = 0.3

# datas
split_indices = []
split_indices_real = []
Xs,Xreals = [],[]
population_names = []
cum_index, cum_index_real = 0,0

for name, group in gen_data.groupby('population'):
    
    print(name)
    # generated data
    print('generated ....')
    item = group.values[:, 2:]
    np.random.seed(21)
    N = int(item.shape[0]*sample_ratio)
    sample_indices = np.random.choice(np.arange(N), size = N, replace = False)
    item = item[sample_indices, :]
    Xs.append(item)
    cum_index += item.shape[0]
    split_indices.append(cum_index)
    
    # real data
    print('real....')
    item_real = final_df[final_df['population'] == name].values[:,1:]
    if N <= item_real.shape[0]:
        sample_indices = np.random.choice(np.arange(N), size = N, replace = False)
        item_real = item_real[sample_indices, :]
    Xreals.append(item_real)
    cum_index_real += item_real.shape[0]
    split_indices_real.append(cum_index_real)
    
    population_names.append(name)

print(split_indices, split_indices_real)
    
# data
X = np.concatenate(Xs, axis=0)
X_real = np.concatenate(Xreals, axis = 0)
print(X.shape, X_real.shape)

In [ ]:
# change population name to different population group to see tsne for different ancestry 
population_name = 'eur'

# show tsne
Xs = np.split(X, split_indices[:-1])
Xreals = np.split(X_real, split_indices_real[:-1])
i = poulation_names.index(population_name)
X_emb_gen = perform_tsne(Xs[i], perplexity = 200, n_components_pca = 25, method = 'pca', scaler = 'standard', seed = 23)
X_emb_real = perform_tsne(Xreals[i], perplexity = 200, n_components_pca = 25, method = 'pca', scaler = 'standard', seed = 23)
plt.figure(figsize=(6, 6))
plt.scatter(X_emb_gen[:, 0], X_emb_gen[:, 1], label=population_names[i], s=50, alpha=0.7, c = 'blue')
plt.scatter(X_emb_real[:, 0], X_emb_real[:, 1], label=population_names[i], s=50, alpha=0.7, c = 'red')

plt.title(f'{population_names[i]} Plot')
plt.xticks([])
plt.yticks([])
plt.show()